In [1]:
import requests, time
from bs4 import BeautifulSoup

### 1. Парсинг

Для работы нам понадобятся отзывы на фильм "Начало" с Кинопоиска. Сначала пропишем несколько функций:

In [5]:
def url_processing(url): #обработка запроса
    session = requests.session()
    time.sleep(1)
    req = session.get(url)
    time.sleep(2)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [6]:
def get_pages(first_link): #получение ссылок страниц
    pages_list = [first_link]
    page = url_processing(first_link)
    for link in page.find_all('a'):
        time.sleep(1)
        actualink = link.get('href')
        if actualink.endswith("page/2/") or actualink.endswith("page/3/"):
            flink = "https://www.kinopoisk.ru"+ actualink
            pages_list.append(flink)
    return pages_list

In [7]:
def get_30revs(pages): #получаем 30 отзывов
    gorb30revs = []
    for page_link in pages:
        page_revs = url_processing(page_link)
        for rev in page_revs.find_all('span', {'class': '_reachbanner_'}):
            time.sleep(1)
            gorb30revs.append(rev.text)
    return gorb30revs

Итак, сначала обрабатываем ссылку с отзывами:

In [8]:
%%time
revs = url_processing("https://www.kinopoisk.ru/film/447301/reviews/")

Wall time: 4.37 s


В обработанной ссылке можно найти отдельно ссылки с хорошими и плохими отзывами:

In [9]:
%%time
good_revs_link = "" #сюда запишется ссылка с хорошими отзывами
bad_revs_link = "" #а сюда — с плохими
for rlink in revs.find_all('a'):
    time.sleep(1)
    gorb_link = rlink.get("href")
    if "good" in gorb_link:
        good_revs_link = "https://www.kinopoisk.ru" + gorb_link
    elif "bad" in gorb_link:
        bad_revs_link = "https://www.kinopoisk.ru" + gorb_link

Wall time: 4min 16s


И в одной, и в другой ссылках на каждой странице по 10 отзывов. Поэтому, в фунцкии get_pages нам достаточно получить по 3 страницы. Начнем с положительных отзывов:

In [10]:
%%time
gpage_urls = list(set(get_pages(good_revs_link))) #использовали set, чтобы избавиться от дубликатов

Wall time: 4min 21s


In [11]:
%%time
g30revs = get_30revs(gpage_urls) #здесь все 30 хороших отзывов

Wall time: 47 s


Теперь перейдем к отрицательным отзывам:

In [12]:
%%time
bpage_urls = list(set(get_pages(bad_revs_link)))

Wall time: 4min 8s


In [13]:
%%time
b30revs = get_30revs(bpage_urls)

Wall time: 46.7 s


Иногда даже с time.sleep() обработка данных прерывается из-за блокировок, поэтому будет неплохо записать однажды полученные отзывы в файлы:

In [17]:
with open("grevs.txt", "w", encoding="utf-8") as f: #положительные отзывы
    for text in g30revs:
        f.write(text)
        f.write("=====") #разделитель

In [18]:
with open("brevs.txt", "w", encoding="utf-8") as f: #отрицательные отзывы
    for text in b30revs:
        f.write(text)
        f.write("=====")

### 2. Работа с текстами из файлов

In [2]:
import nltk

In [3]:
import collections
from collections import Counter

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [116]:
def tknzd_lems(file): #лемматизация, токенизация
    lems = []
    freqlist = Counter()
    with open(file, "r", encoding="utf-8") as f:
        all_revs = f.read().split("=====")
        for text in all_revs:
            for word in nltk.word_tokenize(text.lower()):
                lem = morph.parse(word)[0].normal_form
                lems.append(lem)
    return lems

In [137]:
def set_borg_lems(lems1, lems2): #составление 2 множеств со словами, которые втсречаются только в хороших/плохих отзывах
    onelist_lems = []
    for lem in lems1.items():
        if lem[0] not in lems2:  
            onelist_lems.append(lem)
    return dict(set(onelist_lems))

In [119]:
def fr_more_than2(fr_lems): #избавляемся от лемм, встретившихся меньше 3 раз
    lemset = []
    for lem_fr in fr_lems.items():
        if lem_fr[1] > 2:
            lemset.append(lem_fr)
    return dict(lemset)

In [120]:
glems = tknzd_lems("grevs.txt")

In [121]:
blems = tknzd_lems("brevs.txt")

In [122]:
fr_glems = collections.Counter(glems)
fr_blems = collections.Counter(blems)

In [123]:
glems_mt2 = fr_more_than2(fr_glems)
blems_mt2 = fr_more_than2(fr_blems)       

In [138]:
onlyglems = set_borg_lems(glems_mt2, blems_mt2)

In [139]:
onlyblems = set_borg_lems(blems_mt2, glems_mt2)

In [141]:
bg_freq_lists = {}
bg_freq_lists["good"] = onlyglems
bg_freq_lists["bad"] = onlyblems

### 3. Определялка и ее точность

In [57]:
from sklearn.metrics import accuracy_score

In [147]:
def bad_or_good_detect(freq_lists, text):
    counts = Counter()
    for b_or_g, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[b_or_g] += int(freq_list[word] > 0)
    return counts.most_common()

Закинем все тексты отзывов в словарь с ключами good и bad соответственно:

In [148]:
all_reviews = {}
with open("grevs.txt", "r", encoding="utf-8") as f:
    all_grevs = f.read().split("=====")
    all_reviews["good"] = all_grevs

In [149]:
with open("brevs.txt", "r", encoding="utf-8") as f:
    all_brevs = f.read().split("=====")
    all_reviews["bad"] = all_brevs

In [150]:
results = []  
original_data = []  
for bg_revs in all_reviews.items():
    for rev in bg_revs[1]:
        if len(rev) != 0:
            result = bad_or_good_detect(bg_freq_lists, rev)[0][0]
            results.append(result)
            original_data.append(bg_revs[0])

Считаем accuracy:

In [152]:
accr = accuracy_score(results, original_data)
accr

1.0

На мой взгляд, довольно странно видеть такой точный результат, для лучшего понимания работы accuracy было бы хорошо обработать большее количество текстов. Также стоит поразмышлять, возможно ли было б ускорить работу самого парсинга, например, избежать if/elif.